In [ ]:
import pandas as pd
import numpy as np
import os
import random
random.seed(100)
#os.chdir('/home/sunil/Desktop/datahack')
train = pd.read_csv('train.csv')

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
train.rename(columns={'Loan_Status': 'class'}, inplace=True)

In [ ]:
train.dtypes

In [ ]:
train.isnull().values.any()

In [ ]:
train=train.dropna()

In [ ]:
train.shape

In [ ]:
train.isnull().values.any()

In [ ]:
y = train['class'] 
X = train.drop('class', axis=1)  
X = train.drop('Loan_ID',axis=1)

In [ ]:
for col_name in  X.columns:
    if(X[col_name].dtype == 'object'):
        X[col_name]= X[col_name].astype('category')
        X[col_name] = X[col_name].cat.codes

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
X.apply(lambda x: x.value_counts()).T.stack()

In [17]:
#from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y.values,train_size=0.90, test_size=0.10)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train_array=X_train.values
X_test_array=X_test.values

In [ ]:
print(type(X_train_array))
print(type(X_test_array))

In [ ]:
y_test

In [ ]:
X_test_array

In [ ]:
type(X_test_array)

In [ ]:
X_test

In [ ]:
X_train_array

In [ ]:
X_train

In [ ]:
type(X_train_array)

In [ ]:
type(y_train)

In [ ]:
y_train[1]

In [ ]:
import numpy as np 
np.shape(X_train_array)

In [ ]:
np.shape(X_test_array)

In [ ]:
y_test

In [18]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

In [ ]:
import h2o

from h2o.automl import H2OAutoML

h2o.init()

In [21]:
X_train['class'] = y_train

In [22]:
X_test['class'] = y_test

In [23]:
X_train_h2o = h2o.H2OFrame(X_train)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [24]:
X_test_h2o = h2o.H2OFrame(X_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
X_train_h2o.describe()

In [25]:
features = X_train.columns.values.tolist()
target = "class"

In [26]:
auto_h2o = H2OAutoML(max_runtime_secs=60*3)
auto_h2o.train(x=features,
               y=target,
               training_frame=X_train_h2o)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [27]:
auto_h2o.leaderboard

model_id,auc,logloss,mean_per_class_error,rmse,mse
GBM_grid_1_AutoML_20181212_125527_model_10,0.769781,0.504572,0.285565,0.399134,0.159308
GBM_1_AutoML_20181212_125527,0.751772,0.517613,0.303421,0.407578,0.16612
GBM_grid_1_AutoML_20181212_125527_model_2,0.748044,0.525504,0.288464,0.413998,0.171395
StackedEnsemble_BestOfFamily_AutoML_20181212_125527,0.747047,0.491265,0.293987,0.396147,0.156933
XGBoost_grid_1_AutoML_20181212_125527_model_9,0.746127,0.499134,0.293987,0.399907,0.159925
XGBoost_grid_1_AutoML_20181212_125527_model_10,0.744869,0.491765,0.2815,0.39591,0.156744
StackedEnsemble_AllModels_AutoML_20181212_125527,0.74312,0.491989,0.287744,0.396315,0.157066
GBM_grid_1_AutoML_20181212_125527_model_16,0.742951,0.499916,0.298052,0.401164,0.160933
GBM_2_AutoML_20181212_125527,0.742507,0.513659,0.301825,0.405136,0.164135
GBM_grid_1_AutoML_20181212_125527_model_6,0.742092,0.500145,0.289922,0.399891,0.159913


In [28]:
auto_h2o = auto_h2o.leader

In [29]:
X_test1 = X_test
del X_test1['class']

In [30]:
X_test_h2o1 = h2o.H2OFrame(X_test1)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
X_test_h2o1.describe()

In [31]:
df_test_hat = auto_h2o.predict(X_test_h2o1)
predictions = h2o.as_list(df_test_hat["predict"])

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [ ]:
predictions

In [32]:
y_test=list(y_test)

In [33]:
import sklearn
from sklearn.metrics import accuracy_score
print("Accuracy score", sklearn.metrics.accuracy_score(y_test, predictions))

Accuracy score 0.8333333333333334


In [ ]:
av_test_array_h2o.describe()

In [41]:
test_data = pd.read_csv('test.csv')
test_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [42]:
#train.isnull().values.any()
test_data=test_data.dropna()
for col_name in  test_data.columns:
    if(test_data[col_name].dtype == 'object'):
        test_data[col_name]= test_data[col_name].astype('category')
        test_data[col_name] = test_data[col_name].cat.codes


In [37]:
test_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,0,1,1,0,0,0,5720,0,110.0,360.0,1.0,2
1,1,1,1,1,0,0,3076,1500,126.0,360.0,1.0,2
2,2,1,1,2,0,0,5000,1800,208.0,360.0,1.0,2
4,3,1,0,0,1,0,3276,0,78.0,360.0,1.0,2
5,4,1,1,0,1,1,2165,3422,152.0,360.0,1.0,2


In [43]:
temp=test_data['Loan_ID']
del test_data['Loan_ID']
av_test_array_h2o = h2o.H2OFrame(test_data)
results= auto_h2o.predict(av_test_array_h2o)
results=(h2o.as_list(results["predict"])).iloc[:,0]
final=pd.concat([temp, results], axis=1)
final.rename(columns={'predict': 'Loan_Status'}, inplace=True)
file_name="h2oautoml_final_av_test.csv"
final.to_csv(file_name, sep=',',index=False) #,encoding='utf-8')

Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
